# Yield Curve Model Testing Notebook

This notebook exercises the `YieldCurve` class by:
1. Building a flat dummy curve for SOFR and USD LIBOR.
2. Computing spot discount factors for various tenors.
3. Computing forward rates for overnight and IBOR indices.

---

## 1) Import dependencies

We start by importing the necessary libraries and the `YieldCurve` class.

In [7]:
import pandas as pd
from yield_curve.yield_curve_model import YieldCurve


## 2) Define dummy curve data

Create a small DataFrame with dummy rates for two indices, at 1M, 3M, and 6M tenors.

In [8]:
DUMMY_DATA = [
    ['SOFR-1B',             '1M',  0.0020],
    ['SOFR-1B',             '3M',  0.0025],
    ['SOFR-1B',             '6M',  0.0030],
    ['USD-LIBOR-BBA-3M',    '1M',  0.0040],
    ['USD-LIBOR-BBA-3M',    '3M',  0.00425],
    ['USD-LIBOR-BBA-3M',    '6M',  0.00450],
]
data = pd.DataFrame(DUMMY_DATA, columns=['INDEX','AXIS1','VALUES'])

## 3) Define build methods

Specify how each index should be interpolated; here we use piecewise constant interpolation.

In [9]:
build_methods = [
    {'TARGET':'SOFR-1B',           'INTERPOLATION METHOD':'PIECEWISE_CONSTANT'},
    {'TARGET':'USD-LIBOR-BBA-3M',  'INTERPOLATION METHOD':'PIECEWISE_CONSTANT'},
]


## 4) Instantiate the yield curve

Set the valuation (as‑of) date to June 26, 2025 and build the `YieldCurve` object.

In [10]:
as_of = "2025-06-26"
yc = YieldCurve(as_of, data, build_methods)

## 5) Compute and display spot discount factors

We pick several (index, date) pairs to show discount factors.

In [11]:
print("--- Spot Discount Factors ---")
tests = [
    ("SOFR-1B",           "2025-07-26"),
    ("SOFR-1B",           "2025-09-26"),
    ("USD-LIBOR-BBA-3M",  "2025-07-26"),
    ("USD-LIBOR-BBA-3M",  "2025-09-26"),
]
for idx, dt in tests:
    df = yc.discountFactor(idx, dt)
    print(f"{idx:>18} @ {dt}: DF = {df:.6f}")

--- Spot Discount Factors ---
           SOFR-1B @ 2025-07-26: DF = 0.999833
           SOFR-1B @ 2025-09-26: DF = 0.999420
  USD-LIBOR-BBA-3M @ 2025-07-26: DF = 0.999667
  USD-LIBOR-BBA-3M @ 2025-09-26: DF = 0.998960


## 6) Compute and display forward rates

Finally, we compute 1M and 3M overnight forwards, and a 3M LIBOR forward:

In [12]:
print("--- Forward Rates ---")
fwd_1m    = yc.forward("SOFR-1B",            as_of, "1M")
fwd_3m    = yc.forward("SOFR-1B",            as_of, "3M")
libor_fwd = yc.forward("USD-LIBOR-BBA-3M",     as_of)

print(f"SOFR 1M forward:    {fwd_1m:.6%}")
print(f"SOFR 3M forward:    {fwd_3m:.6%}")
print(f"LIBOR 3M forward:   {libor_fwd:.6%}")


--- Forward Rates ---
SOFR 1M forward:    0.200018%
SOFR 3M forward:    0.227240%
LIBOR 3M forward:   0.407277%
